In [ ]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import os
# from io import StringIO
import io
from PIL import Image
from scipy import misc
import matplotlib.pyplot as plt
import time
import math
import tensorflow as tf
from tensorflow.python.framework import ops
%matplotlib inline